In [4]:
import glob
import re

from PIL import Image
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation,Dropout,Flatten,Dense
from tensorflow.keras.utils import to_categorical

In [5]:
IMG_DATA_DIR = '../data/scores/'
LABEL_DATA = '../data/inning.csv'
IMG_WIDTH = 320
IMG_HEIGHT = 90
img_files = glob.glob(IMG_DATA_DIR + '*.jpg')
label = pd.read_csv(LABEL_DATA)

In [6]:
# creare train data
x_train = []
y_train = []
x_test = []
y_test = []
for i, img_file in enumerate(img_files):
    file_num = re.sub(r'\D', '', img_file)
    label_num = label[label['frame'] == int(file_num)]['inning'].values[0]
    img = Image.open(img_file)
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img = np.asarray(img)

    if i % 5 == 0:
        x_test.append(img)
        y_test.append(label_num)
    else:
        x_train.append(img)
        y_train.append(label_num)

x_train = np.array(x_train)
x_test  = np.array(x_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

In [7]:
x_train.shape

(2572, 90, 320, 3)

In [8]:
# preprocessing
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

In [13]:
# create model
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [14]:
history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2022-11-10 13:49:36.588230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 3s 37ms/step - loss: 1.9098 - accuracy: 0.3247
Epoch 2/10
81/81 [==============================] - 3s 39ms/step - loss: 1.0956 - accuracy: 0.6621
Epoch 3/10
81/81 [==============================] - 3s 34ms/step - loss: 0.4628 - accuracy: 0.8806
Epoch 4/10
81/81 [==============================] - 3s 37ms/step - loss: 0.1807 - accuracy: 0.9611
Epoch 5/10
81/81 [==============================] - 3s 33ms/step - loss: 0.0756 - accuracy: 0.9868
Epoch 6/10
81/81 [==============================] - 3s 33ms/step - loss: 0.0242 - accuracy: 0.9988
Epoch 7/10
81/81 [==============================] - 3s 33ms/step - loss: 0.0137 - accuracy: 1.0000
Epoch 8/10
81/81 [==============================] - 3s 34ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 9/10
81/81 [==============================] - 3s 35ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 10/10
81/81 [==============================] - 3s 33ms/step - loss: 0.0051 - accuracy: 1.0000


In [15]:
model.evaluate(x_test, y_test)

 4/21 [====>.........................] - ETA: 0s - loss: 0.0077 - accuracy: 1.0000

2022-11-10 13:50:07.417569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 1s 32ms/step - loss: 0.0071 - accuracy: 1.0000


[0.0071387579664587975, 1.0]